In [10]:
import spacy
from spacy.lang.en import English
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora
import pickle
import gensim
import pandas as pd

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ryanmcnally/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [6]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [7]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ryanmcnally/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4] #discard short words
    tokens = [token for token in tokens if token not in en_stop] #remove if stop word
    tokens = [get_lemma(token) for token in tokens] #lemmatize each word
    return tokens

In [30]:
dataset = open('nytimes_news_articles.txt').readlines()

In [43]:
ds = []
for line in dataset:
    if line[:4] == 'URL:':
        ''.join(newstring)
        ds.append(newstring)
        newstring = []
    newstring.append(line)

In [44]:
links = [i[0] for i in ds]
ds = [''.join(i[2:]).replace('\n',' ') for i in ds]

In [51]:
text_data = []
for article in ds:
    tokens = prepare_text_for_lda(article)
    text_data.append(tokens)

In [53]:
# creates dictionary generator
dictionary = corpora.Dictionary(text_data)

In [54]:
# creates a list of lists of tuples, with index for each word in bag of words
corpus = [dictionary.doc2bow(text) for text in text_data]

In [55]:
#creates a pickle file and dictionary file to save progress
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [56]:
#LDA Model instantiation , corpus is the list of tuples, dictionary maps the words to indices

NUM_TOPICS = 5 # arbitrary
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [58]:
#LDA Model instantiation , corpus is the list of tuples, dictionary maps the words to indices

NUM_TOPICS = 25 # arbitrary
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model25.gensim')